In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import random
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [2]:
train_df = pd.read_csv("train.csv")
x_train = train_df.drop(["SalePrice"], axis = 1)
y_train = train_df["SalePrice"]
x_test = pd.read_csv('test_toup.csv')

answer_df = pd.read_csv("samples submission.csv")

In [3]:
y_true = np.array(answer_df["SalePrice"])
y_true

array([208500, 181500, 223500, 140000, 250000, 143000, 307000, 200000,
       129900, 118000, 129500, 345000, 144000, 279500, 157000, 132000,
       149000,  90000, 159000, 139000, 325300, 139400, 230000, 129900,
       154000, 256300, 134800, 306000, 207500,  68500,  40000, 149350,
       179900, 165500, 277500, 309000, 145000, 153000, 109000,  82000,
       160000, 170000, 144000, 130250, 141000, 319900, 239686, 249700,
       113000, 127000, 177000, 114500, 110000, 385000, 130000, 180500,
       172500, 196500, 438780, 124900, 158000, 101000, 202500, 140000,
       219500, 317000, 180000, 226000,  80000, 225000, 244000, 129500,
       185000, 144900, 107400,  91000, 135750, 127000, 136500, 110000,
       193500, 153500, 245000, 126500, 168500, 260000, 174000, 164500,
        85000, 123600, 109900,  98600, 163500, 133900, 204750, 185000,
       214000,  94750,  83000, 128950, 205000, 178000, 118964, 198900,
       169500, 250000, 100000, 115000, 115000, 190000, 136900, 180000,
      

In [4]:
x_train.shape

(1321, 80)

In [5]:
x_train["type"] = "train"
x_test["type"] = "test"

In [6]:
x_whole = pd.concat([x_train, x_test], axis = 0, ignore_index=True)
print(x_train.shape)
print(x_test.shape)
print(x_whole.shape)

(1321, 81)
(139, 81)
(1460, 81)


In [7]:
object_dtype = []

for i in x_whole.columns:
    if x_whole[i].dtype == 'object':
        object_dtype.append(i)
        
np.array(object_dtype)

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition', 'type'], dtype='<U13')

In [8]:
x_whole['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [9]:
valid_na_columns = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [10]:
num_but_obj_columns = ['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd','GarageCars', 'Fireplaces', 'GarageYrBlt', 'MoSold', 'YrSold']

In [11]:
#for int and float columns
meanable_cols = []


for i in x_whole.columns:
    if i not in num_but_obj_columns:
        if x_whole[i].dtypes == 'int' or x_whole[i].dtypes == 'float':
            meanable_cols.append(i)
            
meanable_cols

['Id',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [12]:
#384 - year sold
#809 - bsmt
x_whole.drop([809, 384], axis = 0, inplace=True)
y_train.drop([384, 809], axis = 0, inplace = True)
x_whole.reset_index(inplace=True)

In [13]:
le = preprocessing.LabelEncoder()
ohe = OneHotEncoder() 
for i in x_whole.columns:
    if i not in valid_na_columns:
        if i in meanable_cols:
            x_whole[i].fillna(-1, inplace = True)
            for j in range(len(train_df[i])):
                if train_df[i][j] < 0:
                    x_whole[i][j] = random.choice([x_whole[i].mean() - x_whole[i].std(), x_whole[i].mean() + x_whole[i].std()])
        else:
            print(i)
            x_whole[i].fillna("None", inplace = True)
            for j in range(len(x_whole[i])):
                if x_whole[i][j] == "None":
                    x_whole[i][j] = random.choice(x_whole[i].dropna().unique())
            if i in object_dtype:
                x_whole[i] = le.fit_transform(x_whole[i])
                
    else:
        print(i)
        x_whole[i].fillna('NaN', inplace=True)
        x_whole[i] = le.fit_transform(x_whole[i])

index
MSSubClass
MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
BsmtFullBath
BsmtHalfBath
FullBath


/home/rohan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
MoSold
YrSold
SaleType
SaleCondition
type


In [14]:
x_whole['GarageYrBlt'].unique()

array([1997.0, 1934.0, 2005.0, 2000.0, 1999.0, 1963.0, 2004.0, 1931.0,
       2001.0, 1958.0, 1936.0, 1975.0, 2007.0, 1971.0, 1960.0, 1923.0,
       1933.0, 1950.0, 2009.0, 2006.0, 1984.0, 2003.0, 1945.0, 1926.0,
       1976.0, 1955.0, 1981.0, 1991.0, 1987.0, 1961.0, 1986.0, 1988.0,
       1935.0, 1920.0, 1957.0, 1993.0, 1990.0, 1916.0, 1979.0, 1932.0,
       1972.0, 1989.0, 1918.0, 1980.0, 1924.0, 1948.0, 1962.0, 1964.0,
       1922.0, 1996.0, 1977.0, 1940.0, 1939.0, 2002.0, 1995.0, 1974.0,
       1959.0, 1949.0, 2008.0, 1994.0, 1910.0, 1954.0, 1956.0, 1978.0,
       1965.0, 1966.0, 1982.0, 1951.0, 1992.0, 1983.0, 1970.0, 1953.0,
       1968.0, 1998.0, 1925.0, 1941.0, 1930.0, 1985.0, 2010.0, 1927.0,
       1947.0, 1908.0, 1915.0, 1937.0, 1973.0, 1942.0, 1938.0, 1900.0,
       1967.0, 1952.0, 1928.0, 1969.0, 1906.0, 1921.0, 1914.0, 1946.0,
       1929.0], dtype=object)

In [15]:
x_whole['GarageYrBlt'] = x_whole['GarageYrBlt'].astype(float)
x_whole['GarageYrBlt'].unique()

array([1997., 1934., 2005., 2000., 1999., 1963., 2004., 1931., 2001.,
       1958., 1936., 1975., 2007., 1971., 1960., 1923., 1933., 1950.,
       2009., 2006., 1984., 2003., 1945., 1926., 1976., 1955., 1981.,
       1991., 1987., 1961., 1986., 1988., 1935., 1920., 1957., 1993.,
       1990., 1916., 1979., 1932., 1972., 1989., 1918., 1980., 1924.,
       1948., 1962., 1964., 1922., 1996., 1977., 1940., 1939., 2002.,
       1995., 1974., 1959., 1949., 2008., 1994., 1910., 1954., 1956.,
       1978., 1965., 1966., 1982., 1951., 1992., 1983., 1970., 1953.,
       1968., 1998., 1925., 1941., 1930., 1985., 2010., 1927., 1947.,
       1908., 1915., 1937., 1973., 1942., 1938., 1900., 1967., 1952.,
       1928., 1969., 1906., 1921., 1914., 1946., 1929.])

In [16]:
le.inverse_transform(x_whole["type"])

array(['train', 'train', 'train', ..., 'test', 'test', 'test'],
      dtype=object)

In [17]:
x_whole.drop("LotFrontage", axis = 1, inplace = True)


In [18]:
x_whole["new_col_overall"] = x_whole["OverallCond"] + x_whole["OverallQual"]
x_whole["new_col_bsmt"] = x_whole["OverallCond"] + x_whole["OverallQual"]
x_whole.drop(["Condition2", "Exterior2nd"], axis=1, inplace=True)




In [19]:
x_train = x_whole[x_whole["type"] == 1]
x_test = x_whole[x_whole["type"] == 0]
print(x_train.shape)
print(x_test.shape)
x_train.drop("type", axis = 1, inplace=True)
x_test.drop("type", axis = 1, inplace = True)

(1319, 81)
(139, 81)


/home/rohan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
for i in range(len(x_whole)):
    if(x_whole["YearBuilt"][i] > x_whole["YrSold"][i] or x_whole["YrSold"][i] < x_whole["YearRemodAdd"][i]):
        print(i)

In [21]:
# model = XGBClassifier()

In [22]:
# model.fit(x_train, y_train)

In [23]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
y_pred = model.predict(x_test)

In [25]:
y_pred

array([213599.2617764 , 198246.57435135, 216047.39725615, 173276.30065321,
       286939.75368183, 167020.16238809, 286249.50391371, 245498.38397993,
       148989.40530237,  90781.83342456, 123940.45433175, 381680.71620899,
       137190.42120915, 251279.22962478, 155483.09693855, 127127.34121947,
       145960.13752735,  98183.29990536, 149581.4874461 , 109523.7351243 ,
       308920.8441758 , 120855.70774847, 242235.87414024, 131117.06730862,
       151184.31743295, 261441.68743037, 127669.92389654, 276862.65589309,
       193180.03311421,  81001.69089864,  90960.99552503, 126563.64766875,
       211908.93365335, 174860.43620371, 293605.62937305, 309134.9485686 ,
       127777.612227  , 160117.40521876, 114616.6145416 ,  82674.51858358,
       134788.95957297, 152606.55949682, 163991.20527205, 134961.42636616,
       131102.74403998, 299434.29609295, 271382.39144301, 249998.14329698,
        49804.165784  , 125604.23707954, 179980.15175065, 106567.55022993,
        92431.5177217 , 3

In [26]:
df_out = pd.DataFrame(y_pred)

In [27]:
df_out


,0
0,213599.261776
1,198246.574351
2,216047.397256
3,173276.300653
4,286939.753682
...,...
134,158339.295928
135,197155.444488
136,131179.375543
137,166079.637315


In [28]:
df_out.columns = ["SalePrice"]

In [29]:
df_out["Id"] = df_out.index

In [30]:
df_out = df_out[["Id", "SalePrice"]]

In [31]:
df_out


,Id,SalePrice
0,0,213599.261776
1,1,198246.574351
2,2,216047.397256
3,3,173276.300653
4,4,286939.753682
...,...,...
134,134,158339.295928
135,135,197155.444488
136,136,131179.375543
137,137,166079.637315


In [32]:
df_out["Id"] = range(1, 140)
df_out["SalePrice"] = df_out["SalePrice"].astype(int)

In [33]:
df_out.set_index("Id")

,SalePrice
Id,
1,213599
2,198246
3,216047
4,173276
5,286939
...,...
135,158339
136,197155
137,131179


In [34]:
df_out.to_csv("ouput.csv", index = False)

In [35]:
print(np.sqrt(metrics.mean_squared_error(y_true, df_out["SalePrice"])))

22031.580835426077
